In [ ]:
import json

### Apriamo un file json con tutti i risultati del campionato di seria A 2015/16

In [ ]:
fp = open('datasets/serie_A/2015-16.json')
campionato = json.load(fp)

### Ispezioniamo il contenuto "trasformato" in strutture dati Python

In [ ]:
type(campionato)

In [ ]:
campionato.keys()

In [ ]:
campionato['name']

### Ci aspettiamo che rounds corriponda ad una lista ...

In [ ]:
type(campionato['rounds'])

In [ ]:
campionato['rounds'][0]

In [ ]:
campionato['rounds'][0].keys()

In [ ]:
campionato['rounds'][0]['name']

In [ ]:
type(campionato['rounds'][0]['matches'])

In [ ]:
campionato['rounds'][0]['matches'][0]

### Abbiamo la struttura
1. Un campionato è un dizionario con il nome (una stringa) e la lista delle giornate
2. Una giornata (un elemento della lista) è a sua volta un dizinario con il nome (1^ giornata, 2^ giornata, ....) e la lista degli incontri
3. Un incontro è un dizionario con 5 voci: 
   1. data (in cui si dell'incontro), 
   2. team1 (squadra di casa)
   3. team2 (squadra ospite)
   4. score1 (goal segnati dalla squadra di casa)
   5. score2 (goal segnati dalla squadra ospite)
4. Team1 e team2 sono a loro volta dei dizionari

### Controlliamo il numero delle giornate

In [ ]:
num_giornate = len(campionato['rounds'])
print(num_giornate)

### Proviamo innanzitutto a stampare (in modo "pulito") il programma della giornata

In [ ]:
for partita in campionato['rounds'][0]['matches']:
    print(partita['team1']['name'],partita['team2']['name'])

### Primo obiettivo: ottenere una lista di coppie (nomesquadre,codicesquadra), con tutte le squadre partecipanti

#### Soluzioni non-pythoniche

In [ ]:
squadre_partecipanti = []
for partita in campionato['rounds'][0]['matches']:
    squadre_partecipanti.append((partita['team1']['name'],partita['team1']['code']))
    squadre_partecipanti.append((partita['team2']['name'],partita['team2']['code']))

In [ ]:
squadre_partecipanti

#### Un po' meglio ma sempre non pythonica

In [ ]:
def squadre(partita):
    return [(partita['team1']['name'],partita['team1']['code']),\
           (partita['team2']['name'],partita['team2']['code'])]

In [ ]:
squadre(campionato['rounds'][0]['matches'][0])

In [ ]:
squadre_partecipanti = []
for partita in campionato['rounds'][0]['matches']:
    squadre_partecipanti += squadre(partita)

In [ ]:
squadre_partecipanti

### List comprehension

In [ ]:
[squadre(p) for p in campionato['rounds'][0]['matches']]

#### Non è ancora quel che vogliamo: non è una lista di coppie, ma una lista di liste formate ciascuna da due coppie

#### Possiamo usare la funzione reduce (pare non amata da Van Rossum)

In [ ]:
from functools import reduce

In [ ]:
def lsum(x,y):
    return x+y

In [ ]:
reduce(lsum, [squadre(p) for p in campionato['rounds'][0]['matches']])

#### Ancora meglio

In [ ]:
reduce(lambda x,y: x+y, [squadre(p) for p in campionato['rounds'][0]['matches']])

In [ ]:
squadre_partecipanti = reduce(lambda x,y: x+y, [squadre(p) for p in campionato['rounds'][0]['matches']])

In [ ]:
squadre_partecipanti

### Vogliamo ora generare una lista di tuple che descriva tutto il calendario
### Le tuple devono contenere un id progressivo

In [ ]:
campionato['rounds'][0]['matches'][0]

In [ ]:
partita = campionato['rounds'][0]['matches'][0]
id = 1
(id, campionato['name'],campionato['rounds'][0]['name'],'andata',squadre(partita)[0][1],squadre(partita)[1][1])